In [1]:
print("Hello Rag")

Hello Rag


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "llama2"


In [9]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings


if MODEL.startswith("gpt"):
    model = ChatOpenAI(api_key=OPENAI_API_KEY,model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings()


model.invoke("Tell me a joke")


C:\Users\AKASH JADHAV\AppData\Local\Temp\ipykernel_12184\1550016297.py:12: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings()


"\nWhy don't scientists trust atoms? Because they make up everything! 😂"

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser

chain.invoke("Tell me a joke")

'Why was the math book sad?\n\nBecause it had too many problems!'

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("indian_culture.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'producer': 'FPDF 1.81', 'creator': 'PyPDF', 'creationdate': 'D:20200917124537', 'source': 'indian_culture.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1'}, page_content='Indian Culture\nNotes\n 9Indian Culture and Heritage Secondary Course\nMODULE - I\nUnderstanding\nCulture\n2\nINDIAN CULTURE\nH\nave you ever thought of the amazing progress we, as human beings, have made in\nvarious spheres of life, be it language, literature, art and architecture, science or\nreligion? Have you ever wondered how all this has been possible? This happened\nbecause, we did not have to make a fresh beginning each time, but were able to make use\nof and build on the work of past generations. You have never had to bother about having\nto make your own script or creating a new language system for yourself. These are already\ngiven to you which you enjoy as a member of society. Then you build on it by making your\ncontribution or addition which further becomes an asset for the comin

In [7]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know"ArithmeticError

context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context",question="Here is a question")

print(prompt.format(context="Here is some context",question="Here is a question"))


Answer the question based on the context below. If you can't
answer the question, reply "I don't know"ArithmeticError

context: Here is some context

Question: Here is a question



In [8]:
chain = prompt | model | parser

In [11]:
from langchain_community.vectorstores import DocArrayInMemorySearch
vectorstore = DocArrayInMemorySearch.from_documents(pages,embedding=embeddings)


d:\ML\LLM and GenAI\openSource_RAG\ragEnv\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [12]:
retriver = vectorstore.as_retriever()

retriver.invoke("Diversity")

[Document(metadata={'producer': 'FPDF 1.81', 'creator': 'PyPDF', 'creationdate': 'D:20200917124537', 'source': 'indian_culture.pdf', 'total_pages': 9, 'page': 3, 'page_label': '4'}, page_content='Indian Culture\nNotes\nIndian Culture and Heritage Secondary Course 12\nMODULE - I\nUnderstanding\nCulture\nthe eighteenth century. India through the ages has shown a remarkable capacity for\nassimilation of ideas. This has contributed to the variety and richness of our culture.\nAlong with contacts with outside cultures, cultural exchange between different regions of\nIndia has also continued. The Chikan work of Lucknow, Phulkari embrodery of Punjab,\nKantha embroidery of Bengal, Patola of Orissa show a distinct regional flavour. Although\nthe centres in the South, North, East and West of India have their characteristic cultures,\nyet these did not develop in absolute isolation. Inspite of physical barriers, Indians used to\ntravel from one part of the country to another for trade or pilgrima

In [14]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriver,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

chain.invoke({"question":"Diversity in India"})

'India is a country with a rich and diverse culture, with multiple factors contributing to its cultural identity. Here are some of the ways in which diversity manifests in India:\n\n1. Religion: India is home to numerous religions, including Hinduism, Islam, Christianity, Sikhism, Buddhism, and Jainism, among others. Each religion has its own beliefs, practices, and rituals, leading to a diverse religious landscape.\n2. Language: With over 22 official languages and numerous dialects, India is linguistically diverse. The various languages spoken in the country reflect the cultural identities of their speakers.\n3. Region: India is divided into 29 states, each with its own unique culture. For example, the cuisine, dress, and customs of people from the southern state of Kerala differ from those of people in the northern state of Punjab.\n4. Ethnicity: India has a diverse population in terms of ethnicity, with groups such as the Dalits (formerly known as "untouchables"), Adivasis (indigeno

In [15]:
questions = [
    "what is continuity??",
    "what is variety??",
    "what is secular outlook??"
]


for question in questions:
    print(f"Question:{question}")
    print(f"Answer: {chain.invoke({'question':question})}")
    print()

Question:what is continuity??
Answer: Continuity refers to the uninterrupted and consistent existence or occurrence of something over time. In the context of Indian culture, it refers to the enduring character of the civilization despite major changes and upheavals throughout its history. The fact that many aspects of Harappan culture are still practised today, such as worshipping Mother Goddess and Pashupati, is an example of continuity in Indian culture. Additionally, the multistoried buildings in metropolitan cities like Mumbai and Delhi are a departure from the one-storey Harappan houses, but the basic spirit of Indian culture has continued to evolve while discarding what was becoming irrelevant in the modern age.

Question:what is variety??
Answer: In the context of Indian culture, "variety" refers to the diverse nature of the culture. Indian culture is characterized by a wide range of intellectual and social aspects, including languages, religions, rituals, dances, and music. The

In [16]:
for s in chain.stream({"question":"what is purpose of this pdf"}):
    print(s,end="",flush=True)

The purpose of this PDF document appears to be a educational resource for students studying Indian culture and heritage. The document provides an overview of the meaning of culture, its components, and the distinctive features of Indian culture. It also discusses the concept of unity in diversity in the context of Indian culture.

The PDF document includes a variety of topics related to Indian culture, such as:

1. Definition of culture and its components (knowledge, beliefs, arts, morals, laws, customs, etc.)
2. The meaning of material and non-material culture in India
3. The secular character of Indian culture and its significance
4. Distinctive features of Indian culture, such as spirituality and adaptability
5. The concept of unity in diversity in the context of Indian culture and its significance

The document also includes terminal exercises for students to practice their understanding of the topics covered. Overall, the purpose of this PDF document seems to be providing a compre